In [150]:
from collections import deque
from typing import List

class Solution:
    def findLadders(self, beginWord: str, endWord: str, wordList: List[str]) -> List[List[str]]:
        def isAncestor(word: str, node: str, depth: int) -> bool:
            if word == node:
                return True
            if depth == 0:
                return False
            for parent in visitedTree[(node, depth)]:
                return isAncestor(word, parent, depth-1)
        
        # backtracking to find all the paths
        result = []
        def backtrack(path, word, depth):
            if depth == 0:
                result.append(path)
                return
            for parent in visitedTree[(word[0], depth)]:
                backtrack(word + path, [parent], depth-1)
            return
        
        n = len(beginWord)
        if endWord not in wordList:
            return []
        
        wordList.append(beginWord)

        # create a hash table to store all the possible substitutions
        # .e.g. for each possible pattern, store all the words that can
        # be transformed to that pattern by substitution of a single character
        # using '?' to represent the character to be substituted since it's
        # what you would use in a regex pattern
        subs_table = {}
        for word in wordList:
            for i in range(n):
                pattern = word[:i] + '?' + word[i+1:]
                if pattern not in subs_table:
                    subs_table[pattern] = []
                subs_table[pattern].append(word)

        wordQueue = deque()
        visitedTree = {}

        # start from beginWord
        # (word, parent, depth)
        wordQueue.append((beginWord, None, 1))
        min_depth = None

        while wordQueue:
            tup = wordQueue.popleft()
            word = tup[0]
            parent = tup[1]
            depth = tup[2]

            # found all shortest paths
            if min_depth is not None and depth > min_depth:
                break
            if word == endWord:
                min_depth = depth
            
            if visitedTree.get((word, depth)) is None:
                visitedTree[(word, depth)] = []
            parents = visitedTree[(word, depth)]

            # if we've already visited this word/parent pair, skip
            if parent in parents:
                continue
            parents.append(parent)

            for i in range(n):
                pattern = word[:i] + '?' + word[i+1:]
                for next_word in subs_table[pattern]:
                    if isAncestor(next_word, word, depth) == False:
                        wordQueue.append((next_word, word, depth+1))
            
        #print(visitedTree)
        result = []
        if min_depth is not None:
            backtrack([], [endWord], min_depth)
        
        return result


In [165]:
from collections import defaultdict
from typing import List

class Solution:
    def generateGraph(self, wordList: List[str]) -> defaultdict:
        # adjacency list to store the intermediate words
        adj = defaultdict(list)
        for word in wordList:
            for i in range(len(word)):
                pattern = word[:i] + "?" + word[i+1:]
                adj[pattern].append(word)
        return adj

    def findLadders(self, beginWord: str, endWord: str, wordList: List[str]) -> List[List[str]]:
        if endWord not in wordList:
            return []

        adj = self.generateGraph(wordList)

        def BFS(start):
            queue = deque([(start, 0)])
            parents = defaultdict(set)
            depths = defaultdict(int)

            while queue:
                word, distance = queue.popleft()
                for i in range(len(word)):
                    pattern = word[:i] + "?" + word[i+1:]
                    for next_word in adj[pattern]:
                        if next_word not in depths:
                            # if the word hasn't been encountered before,
                            # add it to the queue, update parents and depths
                            depths[next_word] = distance + 1
                            parents[next_word].add(word)
                            queue.append((next_word, distance+1))
                        elif next_word in depths and depths[next_word] == distance + 1:
                            # if the word has been encountered before at the same depth,
                            # update the parents. We don't need to worry about earlier
                            # instances of the word since we're doing a BFS and they 
                            # will have been visited already
                            parents[next_word].add(word)
            return parents

        parents = BFS(beginWord)
        paths = []

        def getPathsDFS(endWord, currentPath):
            if endWord == beginWord:
                paths.append(currentPath[::-1])
                return
            for neighbor in parents[endWord]:
                getPathsDFS(neighbor, currentPath + [neighbor])

        getPathsDFS(endWord, [endWord])

        return paths

In [166]:
# test 1
beginWord = "hit"
endWord = "cog"
wordList = ["hot","dot","dog","lot","log","cog"]
expected = [["hit","hot","dot","dog","cog"],["hit","hot","lot","log","cog"]]
result = Solution().findLadders(beginWord, endWord, wordList)
print("result:\n", result, "\nexpected:\n", expected)
print()

# test 2
beginWord = "hit"
endWord = "cog"
wordList = ["hot","dot","dog","lot","log"]
expected = []
result = Solution().findLadders(beginWord, endWord, wordList)
print("result:\n", result, "\nexpected:\n", expected)
print()

# test 3
beginWord = "hot"
endWord = "dog"
wordList = ["hot","dog"]
expected = []
result = Solution().findLadders(beginWord, endWord, wordList)
print("result:\n", result, "\nexpected:\n", expected)
print()

result:
 [['hit', 'hot', 'dot', 'dog', 'cog'], ['hit', 'hot', 'lot', 'log', 'cog']] 
expected:
 [['hit', 'hot', 'dot', 'dog', 'cog'], ['hit', 'hot', 'lot', 'log', 'cog']]

result:
 [] 
expected:
 []

result:
 [] 
expected:
 []

